In [116]:
from src.Webscraping import Webscraping_TrailData as TD
from src.Webscraping import Webscraping_TrailReports as TR
import requests
from bs4 import BeautifulSoup
from pymongo import MongoClient
import pymongo
import math
import pandas as pd
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count
import numpy as np 

In [122]:
hike_urls = pd.read_csv('Olympic_trails_clean.csv')

In [188]:
hike_urls['url'].unique()

array(['https://www.wta.org/go-hiking/hikes/scatter-creek',
       'https://www.wta.org/go-hiking/hikes/mount-rose',
       'https://www.wta.org/go-hiking/hikes/upper-dungeness-river',
       'https://www.wta.org/go-hiking/hikes/maple-hollow-park',
       'https://www.wta.org/go-hiking/hikes/point-no-point',
       'https://www.wta.org/go-hiking/hikes/mary-e-theler-wetlands-nature-preserve',
       'https://www.wta.org/go-hiking/hikes/guillemot-cove',
       'https://www.wta.org/go-hiking/hikes/kopachuck-state-park-trail',
       'https://www.wta.org/go-hiking/hikes/dungeness-national-wildlife-refuge',
       'https://www.wta.org/go-hiking/hikes/big-creek',
       'https://www.wta.org/go-hiking/hikes/mount-walker',
       'https://www.wta.org/go-hiking/hikes/burfoot-park',
       'https://www.wta.org/go-hiking/hikes/lake-ozette',
       'https://www.wta.org/go-hiking/hikes/hoh-river',
       'https://www.wta.org/go-hiking/hikes/upper-lena-lake',
       'https://www.wta.org/go-hiking/hi

In [125]:
hike_urls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 26 columns):
Coast                    360 non-null int64
Dogs not allowed         360 non-null int64
Established campsites    360 non-null int64
Fall foilage             360 non-null int64
Good for kids            360 non-null int64
Lakes                    360 non-null int64
Mountain views           360 non-null int64
Old growth               360 non-null int64
Ridges/passes            360 non-null int64
Rivers                   360 non-null int64
Summits                  360 non-null int64
Waterfalls               360 non-null int64
Wildflowers/Meadows      360 non-null int64
Wildlife                 360 non-null int64
distance                 276 non-null object
elevation_gain           276 non-null float64
highest_point            276 non-null float64
hike_name                360 non-null object
lat                      360 non-null float64
long                     360 non-null float64
numRe

In [131]:
hike_urls['total_distance'] = hike_urls['distance'].apply(lambda x: total_dst(x))

In [157]:
hike_urls['stars'] = hike_urls['rating'].apply(lambda x: stars(str(x)))

In [154]:

def stars(string):
    '''Extracts number of stars'''
    if 'Nan' in string:
        return 'NaN'
    else:
        lst = string.split()
        return float(lst[0])

In [130]:
def total_dst(string):
    """Extracts mileage from string and calculates total mileage depending on if
    description says one-way or roundtrip."""

    if str(string) == 'nan':
        return np.nan
    else:
        miles = []
        try:
            for s in string.split():
                miles.append(float(s))
        except:
            miles.append(string)
        if 'roundtrip' in string:
            return miles[0]
        elif 'one-way' in string:
            return miles[0]*2
        else:
            return miles[0]

In [166]:
def num_votes(string):
    '''Extracts number of votes'''
    if string ==  np.nan:
        return 'NaN'
    else:
        lst = string.strip('(').split()
        return int(lst[0])

In [177]:
hike_urls['number_votes'][0]

nan

In [164]:
hike_urls['number_votes'].apply(lambda x: num_votes(str(x)))

ValueError: invalid literal for int() with base 10: 'nan'

AttributeError: 'list' object has no attribute 'join'

In [179]:
    mc = pymongo.MongoClient()
    db = mc['wta']
    trail_reports = db['trail_reports']
    raw_html = db['html']
    trail_page_raw_html = db['trail_html']

In [180]:
df = pd.DataFrame(list(trail_reports.find()))

In [181]:
df

,Creator,Date,Report,Trail,Trail_condtions,Votes,_id
0,\nrustysailor,"May 06, 2018",Arrived at the trailhead at about 11am to find...,Mount Rose,"\nBeware of:\n snow, trail condit...",3,5af25573acf3d692490fdf8e
1,\nkilpatjw,"May 05, 2018",Got to the trailhead at around 9am and the lot...,Mount Rose,\nBeware of:\n trail conditions\n...,3,5af25573acf3d692490fdf8f
2,\njalaugh,"May 05, 2018",Shortly after the very well marked summit loop...,Mount Rose,"\nBeware of:\n snow, trail condit...",6,5af25573acf3d692490fdf90
3,\nBecuz Its There,"Apr 25, 2018",Made another mad dash up Rose for training. T...,Mount Rose,"\nBeware of:\n snow, trail condit...",7,5af25573acf3d692490fdf91
4,\nCloud Chaser,"Apr 22, 2018","My wife, dog & I headed up Mt Rose this weeken...",Mount Rose,\nBeware of:\n trail conditions\n...,1,5af25573acf3d692490fdf92
5,\nStepbyStep,"Apr 21, 2018",Lots of snow on the loop. Micro spike and pol...,Mount Rose,\nBeware of:\n snow conditions\n ...,4,5af25573acf3d692490fdf94
6,\ntpioletti,"Apr 21, 2018",Parking lot is pretty small but more space on ...,Mount Rose,None,5,5af25573acf3d692490fdf95
7,\nIrene Grinberg,"Apr 20, 2018","Road has some potholes on the way, definitely ...",Mount Rose,"\nBeware of:\n road, snow & trail...",1,5af25573acf3d692490fdf96
8,\nIrene Grinberg,"Apr 20, 2018","Road has some potholes on the way, definitely ...",Mount Rose,"\nBeware of:\n road, snow & trail...",None,5af25573acf3d692490fdf97
9,\nNicole A,"Apr 19, 2018",Started off this sunny morning heading to the ...,Mount Rose,"\nBeware of:\n snow, trail condit...",5,5af25573acf3d692490fdf98


In [77]:
df['Date_type'] = pd.to_datetime(df['Date'])

In [ ]:
df['Creator'] = df['Creator'].apply(lambda x: x.strip('\n'))

In [183]:
len(df['Trail'].unique())

29

In [184]:
db.trail_reports.count()

57732